## Problem 

1. the tfjs algorithm does not work for the matrix with the shape of more than $3\times3$
    (I was trying to debug but the logic is correct, and I was unable to fix it)
2. the inversed matrix will be printed multiple times both for print out to console and append to info list. 
3. for $3\times 3$ matrix, the running time of using numpy is faster.

preference: https://stackoverflow.com/questions/51805389/how-do-i-invert-a-matrix-in-tensorflow-js

In [7]:
import jp_proxy_widget

In [8]:
w = jp_proxy_widget.JSProxyWidget()

js_file = "https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@2.0.0/dist/tf.min.js"
w.load_js_files([js_file])

In [2]:
import numpy as np

In [4]:
m1 = np.array([1, 3, 3, 1, 4, 3, 1, 3, 4]).reshape(3,3)

In [5]:
m1

array([[1, 3, 3],
       [1, 4, 3],
       [1, 3, 4]])

In [36]:
%%time
m1 = np.random.randint(100,size=(3, 3))
np.linalg.inv(m1) 

CPU times: user 155 µs, sys: 11 µs, total: 166 µs
Wall time: 162 µs


array([[ 0.01894571, -0.00842291, -0.00095301],
       [ 0.00087126, -0.00199011,  0.01142208],
       [-0.00941563,  0.01614508, -0.00815663]])

In [37]:
m1

array([[72, 36, 42],
       [43, 70, 93],
       [ 2, 97, 13]])

In [94]:
np.linalg.det(m1) 

-398806.99999999977

In [38]:
m2 = m1.flatten().tolist()
m2_shape = [3,3]

In [39]:
m2

[72, 36, 42, 43, 70, 93, 2, 97, 13]

In [40]:
info = []
def callback(args):
    info.append(args)

In [41]:
%%time
w.element.test_func(m2, m2_shape)

CPU times: user 1.89 ms, sys: 122 µs, total: 2.01 ms
Wall time: 1.97 ms


['method', <class 'jp_proxy_widget.proxy_widget.CommandMaker'>::140539177653808, 'test_func', <class 'jp_proxy_widget.proxy_widget.LiteralMaker'>::140539189162616, <class 'jp_proxy_widget.proxy_widget.LiteralMaker'>::140539189162672]

In [42]:
np.array(info[0])

array([[ 0.15491877, -0.24157746,  0.11683856],
       [-1.62880468,  2.79243326, -1.40034378],
       [ 1.15435278, -1.97938144,  1.        ]])

In [43]:
np.dot(np.array(info[0]),m1)

array([[ 9.99998078e-01, -6.12437725e-06, -1.44412141e+01],
       [ 5.48362732e-06,  1.00001347e+00,  1.73082028e+02],
       [-1.54972076e-06, -7.15255737e-07, -1.22599657e+02]])

In [30]:
w.js_init("""
element.empty()
element.test_func = function(input_matrix, input_shape){
    // calculate the determinant of a matrix m
    function det(m) {
        return tf.tidy(() => {
            const [r, _] = m.shape
            if (r === 2) {
                const t = m.as1D()
                const a = t.slice([0], [1]).dataSync()[0]
                const b = t.slice([1], [1]).dataSync()[0]
                const c = t.slice([2], [1]).dataSync()[0]
                const d = t.slice([3], [1]).dataSync()[0]
                result = a * d - b * c
                return result

            } else {
                let s = 0;
                //rows = [...Array(r).keys()]
                for (let i = 0; i < r; i++) {
                    console.log(rows,rows.filter(e => e !== i))
                    sub_m = m.gather(tf.tensor1d([...Array(r).keys()].filter(e => e !== i), 'int32'))
                    sli = sub_m.slice([0, 1], [r - 1, r - 1])
                    s += Math.pow(-1, i) * det(sli)
                }
                return s
            }
        })
    }

    // the inverse of the matrix : jordan-gauss method
    function invertM(m) {
        return tf.tidy(() => {
            //if (det(m) === 0) {
            //    return
            //}
            const [r, _] = m.shape
            let inv = m.concat(tf.eye(r), 1)
            rows = [...Array(r).keys()]
            for (let i = 0; i < r; i++) {
                inv = tf.tidy(() => {
                    for (let j = i + 1; j < r; j++) {
                        const elt = inv.slice([j, i], [1, 1]).as1D().asScalar()
                        const pivot = inv.slice([i, i], [1, 1]).as1D().asScalar()
                        let newrow
                        if (elt.dataSync()[0] !== 0) {
                            newrow = inv.gather(tf.tensor1d([i], 'int32')).sub(inv.gather(tf.tensor1d([j], 'int32')).div(elt).mul(pivot)).as1D()
                            const sli = inv.gather(rows.filter(e => e !== j))
                            const arr = []
                            if (j === 0) {
                                arr.push(newrow)
                            }
                            sli.unstack().forEach((t, ind) => {
                                if (ind !== j) {
                                    arr.push(t)
                                } else {
                                    arr.push(newrow)
                                    arr.push(t)
                                }
                            })
                            if (j === r - 1) {
                                arr.push(newrow)
                            }
                            inv = tf.stack(arr)
                        }
                    }
                    return inv
                })
            }
            const trian = tf.unstack(inv)
            len = trian.length
            trian[len - 1] = trian[len - 1].div(trian[len - 1].slice(trian[len - 1].shape[0] - 1, 1).asScalar())
            for (let i = r - 2; i > -1; i--) {
                for (j = r - 1; j > i; j--) {
                    trian[i] = trian[i].sub(trian[j].mul(trian[i].slice(j, 1).asScalar()))
                }
                trian[i] = trian[i].div(trian[i].slice(i, 1).asScalar())
            }
            return tf.split(tf.stack(trian), 2, 1)[1]
        })
    }

    const a = tf.tensor2d(input_matrix, input_shape)
    a.print()
    const i = invertM(a)
    i.print()
    callback(i.arraySync())
}
""",callback = callback)

In [14]:
w.debugging_display()

new error message: ReferenceError: m2 is not defined
